# Hands-on

Q-Learning algoritmasını incelediğimize göre, şimdi onu sıfırdan uygulayalım ve Q-Learning ajanımızı iki ortamda eğitelim:

1. Frozen-Lake-v1 (kaygan olmayan ve kaygan versiyon) ☃️ : burada ajanımızın başlangıç durumundan (S) hedef durumuna (G) sadece donmuş karolar (F) üzerinde yürüyerek ve deliklerden (H) kaçınarak gitmesi gerekecektir.

2. Otonom bir taksinin 🚖 yolcularını A noktasından B noktasına taşımak için bir şehirde gezinmeyi öğrenmesi gerekecektir.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/envs.gif)

## Unit 2: Q-Learning with FrozenLake-v1 ⛄ and Taxi-v3 🚕

### 🎮 Environments:

- [FrozenLake-v1](https://gymnasium.farama.org/environments/toy_text/frozen_lake/)
- [Taxi-v3](https://gymnasium.farama.org/environments/toy_text/taxi/)

### 📚 RL-Library:

- Python and NumPy
- [Gymnasium](https://gymnasium.farama.org/)

## Objectives of this notebook 🏆

Not defterinin sonunda, şunları yapacaksınız:

- Ortam kütüphanesi olan **Gymnasium**'u kullanabilecek 
- Sıfırdan bir Q-Learning ajanını kodlayabilecek 
- Eğitilmiş ajanınızı ve kodunuzu güzel bir video tekrarı ve bir değerlendirme puanı ile **Hub'a** gönderebileceksiniz 🔥.

In [1]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit2/requirements-unit2.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 16.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 72.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 52.7 MB/s eta 0:00:00:00:0100:01
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=125214 sha256=52c9d7bb57d1d17187d889abd319ce2b5c147d071433edeff23922fcc5f27b73
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.1
    Uninstalling PyYAML-6.0.1:
      Successfully uninstalled PyYAML-6.0.1
ERROR: pip's dependency resolver does not currently take into account all t

In [2]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg xvfb
!pip3 install -q pyvirtualdisplay

Get:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]       
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:5 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [25.6 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:7 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3160 kB]
Get:8 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1496 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1256 kB]
Hit:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4018 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3867 kB]
Get:13 http:

Yeni yüklenen kütüphanelerin kullanıldığından emin olmak için bazen notebook çalışma zamanını yeniden başlatmak gerekir. Bir sonraki hücre çalışma zamanını çökmeye zorlayacaktır, bu nedenle tekrar bağlanmanız ve kodu buradan başlayarak çalıştırmanız gerekecektir. Bu hile sayesinde sanal ekranımızı çalıştırabileceğiz.

In [ ]:
import os

os.kill(os.getpid(), 9)

In [3]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Importing Necessary Packages

In [4]:
import os
import numpy as np
import gymnasium as gym
import random
import imageio
import tqdm
import pickle5 as pickle

from tqdm.notebook import tqdm

## Part 1: Frozen Lake ⛄ (non slippery version)

Q-Learning ajanımızı **başlangıç durumundan (S) hedef durumuna (G) sadece donmuş karolar (F) üzerinde yürüyerek ve deliklerden (H)** kaçınarak gitmesi için eğiteceğiz.

İki boyutta çevreye sahip olabiliriz:

- `map_name="4x4"`: 4x4 ızgara versiyonu
- `map_name="8x8"`: 8x8 ızgara sürümü


Ortamın iki modu vardır:

- `is_slippery=False`: Donmuş gölün kaygan olmayan yapısı nedeniyle ajan her zaman **amaçlanan yönde** hareket eder (deterministik).
- `is_slippery=True`: Temsilci, donmuş gölün kaygan doğası nedeniyle **her zaman amaçlanan yönde hareket etmeyebilir** (stokastik).

Şimdilik 4x4 harita ve non-slippery ile basit tutalım.
Ortamın nasıl görselleştirilmesi gerektiğini belirten `render_mode` adlı bir parametre ekliyoruz. Bizim durumumuzda **sonunda ortamın bir videosunu kaydetmek istediğimiz için, render_mode'u rgb_array** olarak ayarlamamız gerekiyor.

"rgb_array" belgesinde açıklandığı gibi: Ortamın mevcut durumunu temsil eden tek bir çerçeve döndürür. Çerçeve, x'e y piksel görüntü için RGB değerlerini temsil eden (x, y, 3) şeklinde bir np.ndarray'dir.

In [5]:
env = gym.make(
    'FrozenLake-v1', desc=None, map_name="4x4", 
    is_slippery=False, render_mode="rgb_array"
)

In [ ]:
# bunun gibi kendi özel ızgaranızı oluşturabilirsiniz:
# desc=["SFFF", "FHFH", "FFFH", "HFFG"]
# gym.make('FrozenLake-v1', desc=desc, is_slippery=True)

In [6]:
# We create our environment with gym.make("<name_of_the_environment>")- `is_slippery=False`: The agent always moves in the intended direction due to the non-slippery nature of the frozen lake (deterministic).
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space", env.observation_space)
print("Sample observation", env.observation_space.sample())  # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Discrete(16)
Sample observation 10


`Observation Space Shape Discrete(16)` ile gözlemin **ajanın mevcut konumunu current_row * ncols + current_col (burada hem satır hem de col 0'dan başlar)** olarak temsil eden bir tamsayı olduğunu görüyoruz.

Örneğin, 4x4 haritasındaki hedef konumu aşağıdaki gibi hesaplanabilir: 3 * 4 + 3 = 15. Olası gözlemlerin sayısı haritanın boyutuna bağlıdır. **Örneğin, 4x4 haritada 16 olası gözlem vardır.**


Örneğin, durum = 0 böyle görünür:

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit2/frozenlake.png)

In [7]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())  # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 2


Eylem uzayı (temsilcinin gerçekleştirebileceği olası eylemler kümesi) 4 eylem ile ayrıktır 🎮: 
- 0: SOLA GİT 
- 1: AŞAĞI GİT 
- 2: SAĞA GİT 
- 3: YUKARI GİT

Ödül fonksiyonu 💰: 
- Hedefe ulaşma: +1 
- Deliğe ulaşma: 0 
- Dondurulmuşa ulaşma: 0

## Create and Initialize the Q-table 🗄️

Q-tablomuzu başlatma zamanı! Kaç satır (durum) ve sütun (eylem) kullanacağımızı bilmek için eylem ve gözlem uzayını bilmemiz gerekir. Değerlerini daha önceden zaten biliyoruz, ancak algoritmamızın farklı ortamlar için genelleştirilebilmesi için bunları programatik olarak elde etmek isteyeceğiz. Gym bize bunu yapmamız için bir yol sunuyor: **env.action_space.n** ve **env.observation_space.n**

In [8]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

There are  16  possible states
There are  4  possible actions


In [9]:
def initialize_q_table(state_space, action_space):
    Qtable = np.zeros((state_space, action_space))
    return Qtable

In [10]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)

## Define the greedy policy

Q-Learning bir **politika dışı** algoritma olduğu için iki politikamız olduğunu unutmayın. Bu, değer fonksiyonunu harekete geçirmek ve güncellemek için **farklı bir politika** kullandığımız anlamına gelir.

- Epsilon-greedy politikası (hareket politikası) 
- Greedy-politikası (güncelleme politikası)

Açgözlü politika aynı zamanda Q-öğrenme ajanı eğitimi tamamladığında sahip olacağımız nihai politika olacaktır. Açgözlü politika, Q-tablosunu kullanarak bir eylem seçmek için kullanılır.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/off-on-4.jpg)

In [11]:
def greedy_policy(Qtable, state):
    action = np.argmax(Qtable[state][:])
    return action

## Define the epsilon-greedy policy

Epsilon-greedy, keşif/sömürü değiş tokuşunu ele alan eğitim politikasıdır.

Epsilon-greedy ile ilgili fikir:

- Olasılık 1 - ɛ* ile: **sömürme** yaparız (yani ajanımız en yüksek durum-eylem çifti değerine sahip eylemi seçer).

- Olasılık ɛ* ile: **keşif** yaparız (rastgele bir eylem deneriz).

Eğitim devam ettikçe, giderek daha az keşfe ve daha fazla sömürüye ihtiyaç duyacağımız için epsilon değerini kademeli olarak **azaltırız**.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-4.jpg)

In [12]:
def epsilon_greedy_policy(Qtable, state, epsilon):
    random_num = random.uniform(0, 1)
    if random_num > epsilon:
        action = greedy_policy(Qtable, state)
    else:
#         action = random.choice([0, 1, 2, 3])
        action = env.action_space.sample()
    
    return action

## Define the hyperparameters

Keşifle ilgili hiperparametreler en önemlilerinden bazılarıdır.

- Temsilcimizin iyi bir değer yaklaşımı öğrenmek için **durum uzayını yeterince keşfettiğinden** emin olmamız gerekir. Bunu yapmak için, epsilonun aşamalı olarak azalması gerekir. 
- Epsilonu çok hızlı azaltırsanız (çok yüksek decay_rate), **ajanınızın takılıp kalması** riskini alırsınız, çünkü ajanınız durum uzayını yeterince keşfetmemiştir ve bu nedenle sorunu çözemez.

In [13]:
# Training parameters
n_training_episodes = 10000  # Total training episodes
learning_rate = 0.7  # Learning rate

# Evaluation parameters
n_eval_episodes = 100  # Total number of test episodes

# Environment parameters
env_id = "FrozenLake-v1"  # Name of the environment
max_steps = 99  # Max steps per episode
gamma = 0.95  # Discounting rate
eval_seed = []  # The evaluation seed of the environment

# Exploration parameters
max_epsilon = 1.0  # Exploration probability at start
min_epsilon = 0.05  # Minimum exploration probability
decay_rate = 0.0005  # Exponential decay rate for exploration prob

## Create the training loop method

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-2.jpg)

```
For episode in the total of training episodes:

Reduce epsilon (since we need less and less exploration)
Reset the environment

  For step in max timesteps:
    Choose the action At using epsilon greedy policy
    Take the action (a) and observe the outcome state(s') and reward (r)
    Update the Q-value Q(s,a) using Bellman equation Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    If done, finish the episode
    Our next state is the new state
```

In [14]:
def train(n_training_episodes, min_epsilon, max_epsilon, 
          decay_rate, env, max_steps, Qtable):
    for episode in tqdm(range(n_training_episodes)):
        # Reduce epsilon (because we need less and less exploration)
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

        # Reset the environment
        state, info = env.reset()
        step = 0
        termiated = False
        truncated = False

        for step in range(max_steps):
            action = epsilon_greedy_policy(Qtable, state, epsilon)
            new_state, reward, terminated, truncated, info = env.step(action)
            Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

            if terminated or truncated:
                break

            state = new_state
    
    return Qtable

In [15]:
Qtable_frozenlake = train(
    n_training_episodes, min_epsilon, max_epsilon, 
    decay_rate, env, max_steps, Qtable_frozenlake
)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [16]:
Qtable_frozenlake

array([[0.73509189, 0.77378094, 0.77378094, 0.73509189],
       [0.73509189, 0.        , 0.81450625, 0.77378094],
       [0.77378094, 0.857375  , 0.77378094, 0.81450625],
       [0.81450625, 0.        , 0.77378092, 0.77378093],
       [0.77378094, 0.81450625, 0.        , 0.73509189],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9025    , 0.        , 0.81450625],
       [0.        , 0.        , 0.        , 0.        ],
       [0.81450625, 0.        , 0.857375  , 0.77378094],
       [0.81450625, 0.9025    , 0.9025    , 0.        ],
       [0.857375  , 0.95      , 0.        , 0.857375  ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9025    , 0.95      , 0.857375  ],
       [0.9025    , 0.95      , 1.        , 0.9025    ],
       [0.        , 0.        , 0.        , 0.        ]])

## The evaluation method 📝

In [17]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
    """
    Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
    :param env: The evaluation environment
    :param n_eval_episodes: Number of episode to evaluate the agent
    :param Q: The Q-table
    :param seed: The evaluation seed array (for taxi-v3)
    """
    episode_rewards = []
    for episode in tqdm(range(n_eval_episodes)):
        if seed:
            state, info = env.reset(seed=seed[episode])
        else:
            state, info = env.reset()
        step = 0
        truncated = False
        terminated = False
        total_rewards_ep = 0

        for step in range(max_steps):
            # Take the action (index) that have the maximum expected future reward given that state
            action = greedy_policy(Q, state)
            new_state, reward, terminated, truncated, info = env.step(action)
            total_rewards_ep += reward

            if terminated or truncated:
                break
            state = new_state
        episode_rewards.append(total_rewards_ep)
    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)

    return mean_reward, std_reward

In [18]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_frozenlake, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

  0%|          | 0/100 [00:00<?, ?it/s]

Mean_reward=1.00 +/- 0.00


## Publish Trained Model on Hub

In [19]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json

In [20]:
def record_video(env, Qtable, out_directory, fps=1):
    """
    Generate a replay video of the agent
    :param env
    :param Qtable: Qtable of our agent
    :param out_directory
    :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
    """
    images = []
    terminated = False
    truncated = False
    state, info = env.reset(seed=random.randint(0, 500))
    img = env.render()
    images.append(img)
    while not terminated or truncated:
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(Qtable[state][:])
        state, reward, terminated, truncated, info = env.step(
            action
        )  # We directly put next_state = state for recording logic
        img = env.render()
        images.append(img)
    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [21]:
def push_to_hub(repo_id, model, env, video_fps=1, local_repo_path="hub"):
    """
    Evaluate, Generate a video and Upload a model to Hugging Face Hub.
    This method does the complete pipeline:
    - It evaluates the model
    - It generates the model card
    - It generates a replay video of the agent
    - It pushes everything to the Hub

    :param repo_id: repo_id: id of the model repository from the Hugging Face Hub
    :param env
    :param video_fps: how many frame per seconds to record our video replay
    (with taxi-v3 and frozenlake-v1 we use 1)
    :param local_repo_path: where the local repository is
    """
    _, repo_name = repo_id.split("/")

    eval_env = env
    api = HfApi()

    # Step 1: Create the repo
    repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
    )

    # Step 2: Download files
    repo_local_path = Path(snapshot_download(repo_id=repo_id))

    # Step 3: Save the model
    if env.spec.kwargs.get("map_name"):
        model["map_name"] = env.spec.kwargs.get("map_name")
        if env.spec.kwargs.get("is_slippery", "") == False:
            model["slippery"] = False

    # Pickle the model
    with open((repo_local_path) / "q-learning.pkl", "wb") as f:
        pickle.dump(model, f)

    # Step 4: Evaluate the model and build JSON with evaluation metrics
    mean_reward, std_reward = evaluate_agent(
        eval_env, model["max_steps"], model["n_eval_episodes"], model["qtable"], model["eval_seed"]
    )

    evaluate_data = {
        "env_id": model["env_id"],
        "mean_reward": mean_reward,
        "n_eval_episodes": model["n_eval_episodes"],
        "eval_datetime": datetime.datetime.now().isoformat(),
    }

    # Write a JSON file called "results.json" that will contain the
    # evaluation results
    with open(repo_local_path / "results.json", "w") as outfile:
        json.dump(evaluate_data, outfile)

    # Step 5: Create the model card
    env_name = model["env_id"]
    if env.spec.kwargs.get("map_name"):
        env_name += "-" + env.spec.kwargs.get("map_name")

    if env.spec.kwargs.get("is_slippery", "") == False:
        env_name += "-" + "no_slippery"

    metadata = {}
    metadata["tags"] = [env_name, "q-learning", "reinforcement-learning", "custom-implementation"]

    # Add metrics
    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name="reinforcement-learning",
        task_id="reinforcement-learning",
        metrics_pretty_name="mean_reward",
        metrics_id="mean_reward",
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name,
    )

    # Merges both dictionaries
    metadata = {**metadata, **eval}

    model_card = f"""
  # **Q-Learning** Agent playing1 **{env_id}**
  This is a trained model of a **Q-Learning** agent playing **{env_id}** .

  ## Usage

  model = load_from_hub(repo_id="{repo_id}", filename="q-learning.pkl")

  # Don't forget to check if you need to add additional attributes (is_slippery=False etc)
  env = gym.make(model["env_id"])
  """

    evaluate_agent(env, model["max_steps"], model["n_eval_episodes"], model["qtable"], model["eval_seed"])

    readme_path = repo_local_path / "README.md"
    readme = ""
    print(readme_path.exists())
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
            readme = f.read()
    else:
        readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
        f.write(readme)

    # Save our metrics to Readme metadata
    metadata_save(readme_path, metadata)

    # Step 6: Record a video
    video_path = repo_local_path / "replay.mp4"
    record_video(env, model["qtable"], video_path, video_fps)

    # Step 7. Push everything to the Hub
    api.upload_folder(
        repo_id=repo_id,
        folder_path=repo_local_path,
        path_in_repo=".",
    )

    print("Your model is pushed to the Hub. You can view your model here: ", repo_url)

In [22]:
from huggingface_hub import notebook_login

notebook_login()

In [23]:
model = {
    "env_id": env_id,
    "max_steps": max_steps,
    "n_training_episodes": n_training_episodes,
    "n_eval_episodes": n_eval_episodes,
    "eval_seed": eval_seed,
    "learning_rate": learning_rate,
    "gamma": gamma,
    "max_epsilon": max_epsilon,
    "min_epsilon": min_epsilon,
    "decay_rate": decay_rate,
    "qtable": Qtable_frozenlake,
}

In [24]:
model

{'env_id': 'FrozenLake-v1',
 'max_steps': 99,
 'n_training_episodes': 10000,
 'n_eval_episodes': 100,
 'eval_seed': [],
 'learning_rate': 0.7,
 'gamma': 0.95,
 'max_epsilon': 1.0,
 'min_epsilon': 0.05,
 'decay_rate': 0.0005,
 'qtable': array([[0.73509189, 0.77378094, 0.77378094, 0.73509189],
        [0.73509189, 0.        , 0.81450625, 0.77378094],
        [0.77378094, 0.857375  , 0.77378094, 0.81450625],
        [0.81450625, 0.        , 0.77378092, 0.77378093],
        [0.77378094, 0.81450625, 0.        , 0.73509189],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.9025    , 0.        , 0.81450625],
        [0.        , 0.        , 0.        , 0.        ],
        [0.81450625, 0.        , 0.857375  , 0.77378094],
        [0.81450625, 0.9025    , 0.9025    , 0.        ],
        [0.857375  , 0.95      , 0.        , 0.857375  ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.

In [25]:
username = "Leotrim"
repo_name = "q-FrozenLake-v1-4x4-noSlippery"
push_to_hub(repo_id=f"{username}/{repo_name}", model=model, env=env)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

False


q-learning.pkl:   0%|          | 0.00/914 [00:00<?, ?B/s]

Your model is pushed to the Hub. You can view your model here:  https://huggingface.co/Leotrim/q-FrozenLake-v1-4x4-noSlippery


## Part 2: Taxi-v3 🚖

Taxi-v3 🚕'te, ızgara dünyasında R(ed), G(reen), Y(ellow) ve B(lue) ile gösterilen dört belirlenmiş konum vardır.

Bölüm başladığında, taksi rastgele bir karede başlar ve yolcu rastgele bir konumdadır. Taksi yolcunun bulunduğu yere gider, yolcuyu alır, yolcunun varış noktasına gider (belirlenen dört konumdan biri) ve sonra yolcuyu bırakır. Yolcu bırakıldıktan sonra bölüm sona erer.

In [26]:
env = gym.make("Taxi-v3", render_mode="rgb_array")

In [27]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

There are  500  possible states
There are  6  possible actions


25 taksi pozisyonu, yolcunun 5 olası konumu (yolcunun takside olduğu durum dahil) ve 4 varış noktası olduğu için 500 ayrık durum vardır.

Eylem uzayı (temsilcinin gerçekleştirebileceği olası eylemler kümesi) ayrıktır ve **6 eylem mevcuttur**:

- 0: güneye hareket et
- 1: kuzeye hareket et
- 2: doğuya hareket et
- 3: Batıya hareket et
- 4: pikap yolcu
- 5: Yolcu bırakma

Ödül işlevi:

- Başka bir ödül tetiklenmediği sürece adım başına -1.
- +20 yolcu teslim etme.
- -10 "alma" ve "bırakma" eylemlerini yasa dışı olarak gerçekleştirme.

In [28]:
Qtable_taxi = initialize_q_table(state_space, action_space)
print(Qtable_taxi)
print("Q-table shape: ", Qtable_taxi.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Q-table shape:  (500, 6)


## Define the hyperparameters ⚙️

In [29]:
# Training parameters
n_training_episodes = 25000  # Total training episodes
learning_rate = 0.7  # Learning rate

# Evaluation parameters
n_eval_episodes = 100  # Total number of test episodes

# DO NOT MODIFY EVAL_SEED
eval_seed = [
    16,
    54,
    165,
    177,
    191,
    191,
    120,
    80,
    149,
    178,
    48,
    38,
    6,
    125,
    174,
    73,
    50,
    172,
    100,
    148,
    146,
    6,
    25,
    40,
    68,
    148,
    49,
    167,
    9,
    97,
    164,
    176,
    61,
    7,
    54,
    55,
    161,
    131,
    184,
    51,
    170,
    12,
    120,
    113,
    95,
    126,
    51,
    98,
    36,
    135,
    54,
    82,
    45,
    95,
    89,
    59,
    95,
    124,
    9,
    113,
    58,
    85,
    51,
    134,
    121,
    169,
    105,
    21,
    30,
    11,
    50,
    65,
    12,
    43,
    82,
    145,
    152,
    97,
    106,
    55,
    31,
    85,
    38,
    112,
    102,
    168,
    123,
    97,
    21,
    83,
    158,
    26,
    80,
    63,
    5,
    81,
    32,
    11,
    28,
    148,
]  # Evaluation seed, this ensures that all classmates agents are trained on the same taxi starting position
# Each seed has a specific starting state

# Environment parameters
env_id = "Taxi-v3"  # Name of the environment
max_steps = 99  # Max steps per episode
gamma = 0.95  # Discounting rate

# Exploration parameters
max_epsilon = 1.0  # Exploration probability at start
min_epsilon = 0.05  # Minimum exploration probability
decay_rate = 0.005  # Exponential decay rate for exploration prob

## Train

In [30]:
Qtable_taxi = train(
    n_training_episodes, min_epsilon, max_epsilon, 
    decay_rate, env, max_steps, Qtable_taxi
)
Qtable_taxi

  0%|          | 0/25000 [00:00<?, ?it/s]

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  2.75200369,   3.94947757,   2.75200369,   3.94947757,
          5.20997639,  -5.05052243],
       [  7.93349125,   9.40367562,   5.2099758 ,   9.40365143,
         10.9512375 ,   0.40367562],
       ...,
       [ 10.9512373 ,  12.58025   ,  10.9512375 ,   9.40367561,
          1.95123746,   1.9512375 ],
       [ -4.42194461,   6.53681725,   4.35298331,  -4.35891922,
        -13.22160565, -12.14572498],
       [ -0.91      ,  -0.973     ,  11.00822128,  18.        ,
          4.91822128,   0.95421971]])

In [31]:
model = {
    "env_id": env_id,
    "max_steps": max_steps,
    "n_training_episodes": n_training_episodes,
    "n_eval_episodes": n_eval_episodes,
    "eval_seed": eval_seed,
    "learning_rate": learning_rate,
    "gamma": gamma,
    "max_epsilon": max_epsilon,
    "min_epsilon": min_epsilon,
    "decay_rate": decay_rate,
    "qtable": Qtable_taxi,
}

In [32]:
username = "Leotrim"
repo_name = "Taxi-v3"
push_to_hub(repo_id=f"{username}/{repo_name}", model=model, env=env)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

False


[swscaler @ 0x64903c0] Warning: data is not aligned! This can lead to a speed loss


q-learning.pkl:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

Your model is pushed to the Hub. You can view your model here:  https://huggingface.co/Leotrim/Taxi-v3
